This notebook contains an example for teaching.

# Testing the Convergence Hypothesis

## Introduction

We provide an additional empirical example of partialling-out with Lasso to estimate the regression coefficient $\beta_1$ in the high-dimensional linear regression model:
  $$
  Y = \beta_1 D +  \beta_2'W + \epsilon.
  $$
  
Specifically, we are interested in how the rates  at which economies of different countries grow ($Y$) are related to the initial wealth levels in each country ($D$) controlling for country's institutional, educational, and other similar characteristics ($W$).
  
The relationship is captured by $\beta_1$, the *speed of convergence/divergence*, which measures the speed at which poor countries catch up $(\beta_1< 0)$ or fall behind $(\beta_1> 0)$ rich countries, after controlling for $W$. Our inference question here is: do poor countries grow faster than rich countries, controlling for educational and other characteristics? In other words, is the speed of convergence negative: $ \beta_1 <0?$ This is the Convergence Hypothesis predicted by the Solow Growth Model. This is a structural economic model. Under some strong assumptions, that we won't state here, the predictive exercise we are doing here can be given causal interpretation.


The outcome $Y$ is the realized annual growth rate of a country's wealth  (Gross Domestic Product per capita). The target regressor ($D$) is the initial level of the country's wealth. The target parameter $\beta_1$ is the speed of convergence, which measures the speed at which poor countries catch up with rich countries. The controls ($W$) include measures of education levels, quality of institutions, trade openness, and political stability in the country.

## Data analysis


We consider the data set GrowthData which is included in the package *hdm*. First, let us load the data set to get familiar with the data.

In [166]:
import hdmpy
import pandas as pd
import numpy as np
import pyreadr
import math
import matplotlib.pyplot as plt
import random

In [167]:
growth_read = pyreadr.read_r("C:/Users/MSI-NB/Documents/GitHub/14.38_Causal_ML/data/GrowthData.RData")
# Extracting the data frame from rdata_read
growth = growth_read[ 'GrowthData' ]

In [168]:
growth

Outcome  intercept  gdpsh465   bmp1l    freeop   freetar    h65   hm65  \
0  -0.024336          1  6.591674  0.2837  0.153491  0.043888  0.007  0.013   
1   0.100473          1  6.829794  0.6141  0.313509  0.061827  0.019  0.032   
2   0.067051          1  8.895082  0.0000  0.204244  0.009186  0.260  0.325   
3   0.064089          1  7.565275  0.1997  0.248714  0.036270  0.061  0.070   
4   0.027930          1  7.162397  0.1740  0.299252  0.037367  0.017  0.027   
..       ...        ...       ...     ...       ...       ...    ...    ...   
85  0.031196          1  8.991064  0.0000  0.371898  0.014586  0.255  0.336   
86  0.034096          1  8.025189  0.0050  0.296437  0.013615  0.108  0.117   
87  0.046900          1  9.030137  0.0000  0.265778  0.008629  0.288  0.337   
88  0.039773          1  8.865312  0.0000  0.282939  0.005048  0.188  0.236   
89  0.040642          1  8.912339  0.0000  0.150366  0.024377  0.257  0.338   

     hf65   p65  ...  seccf65  syr65  syrm65  syrf65  teapri65  teasec65  \
0   0.001  0.29  ...     0.04  0.033   0.057   0.010      47.6      17.3   
1   0.007  0.91  ...     0.64  0.173   0.274   0.067      57.1      18.0   
2   0.201  1.00  ...    18.14  2.573   2.478   2.667      26.5      20.7   
3   0.051  1.00  ...     2.63  0.438   0.453   0.424      27.8      22.7   
4   0.007  0.82  ...     2.11  0.257   0.287   0.229      34.5      17.6   
..    ...   ...  ...      ...    ...     ...     ...       ...       ...   
85  0.170  0.98  ...    11.41  2.226   2.494   1.971      27.5      15.9   
86  0.093  1.00  ...     1.95  0.510   0.694   0.362      20.2      15.7   
87  0.237  1.00  ...    25.64  2.727   2.664   2.788      20.4       9.4   
88  0.139  1.00  ...    10.76  1.888   1.920   1.860      20.0      16.0   
89  0.215  1.00  ...    24.40  3.051   3.235   2.875      18.5      29.1   

       ex1     im1    xr65      tot1  
0   0.0729  0.0667   0.348 -0.014727  
1   0.0940  0.1438   0.525  0.005750  
2   0.1741  0.1750   1.082 -0.010040  
3   0.1265  0.1496   6.625 -0.002195  
4   0.1211  0.1308   2.500  0.003283  
..     ...     ...     ...       ...  
85  0.4407  0.4257   2.529 -0.011883  
86  0.1669  0.2201  25.553 -0.039080  
87  0.3238  0.3134   4.152  0.005175  
88  0.1845  0.1940   0.452 -0.029551  
89  0.1876  0.2007   0.886 -0.036482  

[90 rows x 63 columns]

We determine the dimension of our data set.

In [169]:
growth.shape

(90, 63)

The sample contains $90$ countries and $63$ controls. Thus $p \approx 60$, $n=90$ and $p/n$ is not small. We expect the least squares method to provide a poor estimate of $\beta_1$.  We expect the method based on partialling-out with Lasso to provide a high quality estimate of $\beta_1$.

To check this hypothesis, we analyze the relation between the output variable $Y$ and the other country's characteristics by running a linear regression in the first step.

In [170]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [171]:
# We create the main variables
y = growth['Outcome']
X = growth.drop('Outcome', 1)

In [172]:
# OLS regression
reg_ols  = sm.OLS(y, X).fit()
print(reg_ols.summary())

                            OLS Regression Results                            
Dep. Variable:                Outcome   R-squared:                       0.887
Model:                            OLS   Adj. R-squared:                  0.641
Method:                 Least Squares   F-statistic:                     3.607
Date:                Tue, 06 Apr 2021   Prob (F-statistic):           0.000200
Time:                        15:05:15   Log-Likelihood:                 238.24
No. Observations:                  90   AIC:                            -352.5
Df Residuals:                      28   BIC:                            -197.5
Df Model:                          61                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.2472      0.785      0.315      0.7

In [173]:
# output: estimated regression coefficient corresponding to the target regressor
est_ols = reg_ols.summary2().tables[1]['Coef.']['gdpsh465']

# output: std. error
std_ols = reg_ols.summary2().tables[1]['Std.Err.']['gdpsh465']

# output: 95% confidence interval
lower_ci = reg_ols.summary2().tables[1]['[0.025']['gdpsh465']
upper_ci = reg_ols.summary2().tables[1]['0.975]']['gdpsh465']

## Summarize OLS results

In [178]:
table_1 = np.zeros( (1, 4) )

table_1[0,0] = est_ols  
table_1[0,1] = std_ols   
table_1[0,2] = lower_ci
table_1[0,3] = upper_ci    


table_1_pandas = pd.DataFrame( table_1, columns = [ "Estimator","Std. Error", "lower bound CI", "upper bound CI"  ])
table_1_pandas.index = [ "OLS" ]
table_1_html = table_1_pandas.to_html()
table_1_pandas

Estimator  Std. Error  lower bound CI  upper bound CI
OLS  -0.009378    0.029888         -0.0706        0.051844

Least squares provides a rather noisy estimate (high standard error) of the
speed of convergence, and does not allow us to answer the question
about the convergence hypothesis since the confidence interval includes zero.

In contrast, we can use the partialling-out approach based on lasso regression ("Double Lasso").

In [179]:
# Create main variables
Y = growth['Outcome']
W = growth.drop(['Outcome','intercept', 'gdpsh465'], 1 )
D = growth['gdpsh465']

In [180]:
from sklearn import linear_model

# Seat values for Lasso
lasso_model = linear_model.Lasso( alpha = 0.00077 )

# Part. out Y
Y_lasso_fitted = lasso_model.fit( W, Y ).predict( W )
r_Y = Y - Y_lasso_fitted
r_Y

# Rename pandas series
r_Y = r_Y.rename('r_Y')


# Part. out d
D_lasso_fitted = lasso_model.fit( W, D ).predict( W )
r_D = D - D_lasso_fitted
r_D

# Rename pandas series
r_D = r_D.rename('r_D')

C:\Users\MSI-NB\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 0.0058288944773880885, tolerance: 2.3434976975716032e-05
  model = cd_fast.enet_coordinate_descent(
C:\Users\MSI-NB\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:529: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 2.99724470007767, tolerance: 0.007147912790119585
  model = cd_fast.enet_coordinate_descent(


In [181]:
# Regress residuales
partial_lasso_fit = sm.OLS(r_Y, r_D).fit()
partial_lasso_est = partial_lasso_fit.summary2().tables[1]['Coef.']['r_D']

print( f"Coefficient for D via partialling-out using lasso {partial_lasso_est}" )

Coefficient for D via partialling-out using lasso -0.04774655653302118


In [182]:
# output: estimated regression coefficient corresponding to the target regressor
est_lasso = partial_lasso_fit.summary2().tables[1]['Coef.']['r_D']

# output: std. error
std_lasso = partial_lasso_fit.summary2().tables[1]['Std.Err.']['r_D']

# output: 95% confidence interval
lower_ci_lasso = partial_lasso_fit.summary2().tables[1]['[0.025']['r_D']
upper_ci_lasso = partial_lasso_fit.summary2().tables[1]['0.975]']['r_D']

## Summary LASSO results


Finally, let us have a look at the results.

In [183]:
table_2 = np.zeros( (1, 4) )

table_2[0,0] = est_lasso  
table_2[0,1] = std_lasso   
table_2[0,2] = lower_ci_lasso
table_2[0,3] = upper_ci_lasso    


table_2_pandas = pd.DataFrame( table_2, columns = [ "Estimator","Std. Error", "lower bound CI", "upper bound CI"  ])
table_2_pandas.index = [ "LASSO" ]
table_2_pandas

Estimator  Std. Error  lower bound CI  upper bound CI
LASSO  -0.047747    0.017705       -0.082926       -0.012567

In [184]:
table_3 = table_1_pandas.append(table_2_pandas)
table_3

Estimator  Std. Error  lower bound CI  upper bound CI
OLS    -0.009378    0.029888       -0.070600        0.051844
LASSO  -0.047747    0.017705       -0.082926       -0.012567

In [185]:
table_3_html = table_3.to_html()
print(table_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Estimator</th>
      <th>Std. Error</th>
      <th>lower bound CI</th>
      <th>upper bound CI</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>OLS</th>
      <td>-0.009378</td>
      <td>0.029888</td>
      <td>-0.070600</td>
      <td>0.051844</td>
    </tr>
    <tr>
      <th>LASSO</th>
      <td>-0.047747</td>
      <td>0.017705</td>
      <td>-0.082926</td>
      <td>-0.012567</td>
    </tr>
  </tbody>
</table>


The least square method provides a rather noisy estimate of the speed of convergence. We can not answer the question if poor countries grow faster than rich countries. The least square method does not work when the ratio $p/n$ is large.

In sharp contrast, partialling-out via Lasso provides a more precise estimate. The Lasso based point estimate is $-5\%$ and the $95\%$ confidence interval for the (annual) rate of convergence $[-7.8\%,-2.2\%]$ only includes negative numbers. This empirical evidence does support the convergence hypothesis.

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Estimator</th>
      <th>Std. Error</th>
      <th>lower bound CI</th>
      <th>upper bound CI</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>OLS</th>
      <td>-0.009378</td>
      <td>0.029888</td>
      <td>-0.070600</td>
      <td>0.051844</td>
    </tr>
    <tr>
      <th>LASSO</th>
      <td>-0.047747</td>
      <td>0.017705</td>
      <td>-0.082926</td>
      <td>-0.012567</td>
    </tr>
  </tbody>
</table>